In [ ]:
Comment and Unit Test Generater 

The requirement: 
* use an LLM to generate docstring and comments for Python code
* use an LLM to generate unit test

This is my week 4 day 5 project.

In [24]:
# imports

import os
import io
import sys
import json
import requests
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess
from huggingface_hub import login, InferenceClient
from transformers import AutoTokenizer

In [3]:
# environment

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')

In [4]:
# initialize

openai = OpenAI()
claude = anthropic.Anthropic()
OPENAI_MODEL = "gpt-4o"
CLAUDE_MODEL = "claude-3-5-sonnet-20240620"

In [25]:
hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [28]:
code_qwen = "Qwen/CodeQwen1.5-7B-Chat"
CODE_QWEN_URL = "https://g39mbjooiiwkbgyz.us-east-1.aws.endpoints.huggingface.cloud"

In [49]:
def system_prompt_for_comment():
    system="""
    You are a Python documentation expert. When writing documentation:
    - Follow PEP 257 and Google docstring style guidelines
    - Write clear, concise explanations
    - Include practical examples
    - Highlight edge cases and limitations
    - Use type hints in docstrings
    - Add inline comments only for complex logic
    - Never skip documenting parameters or return values
    - Validate that all documentation is accurate and complete
    """
    return system

In [50]:
def system_prompt_for_unit_test():
    system="""
    You are an expert Python testing engineer who specializes in creating comprehensive unit tests. Follow these principles:
    - Use pytest as the testing framework
    - Follow the Arrange-Act-Assert pattern
    - Test both valid and invalid inputs
    - Include edge cases and boundary conditions
    - Write descriptive test names that explain the scenario being tested
    - Create independent tests that don't rely on each other
    - Use appropriate fixtures and parametrize when needed
    - Add clear comments explaining complex test logic
    - Cover error cases and exceptions
    - Achieve high code coverage while maintaining meaningful tests
    """
    return system

In [51]:
def user_prompt_for_comment(code):
    user = f"""
        Please document this Python code with:
        
        1. A docstring containing:
        - A clear description of purpose and functionality
        - All parameters with types and descriptions
        - Return values with types
        - Exceptions that may be raised
        - At least one usage example
        - Any important notes or limitations
        
        2. Strategic inline comments for:
        - Complex algorithms or business logic
        - Non-obvious implementation choices
        - Performance considerations
        - Edge cases
        
        Here's the code to document:
        \n{code}
        """
    return user;

In [52]:
def user_prompt_for_unit_test(code):
    user = f"""
        Please generate unit tests for the following Python code. Include:
        
        1. Test cases for:
        - Normal/expected inputs
        - Edge cases and boundary values
        - Invalid inputs and error conditions
        - Different combinations of parameters
        - All public methods and functions
        
        2. For each test:
        - Clear test function names describing the scenario
        - Setup code (fixtures if needed)
        - Test data preparation
        - Expected outcomes
        - Assertions checking results
        - Comments explaining complex test logic
        
        3. Include any necessary:
        - Imports
        - Fixtures
        - Mock objects
        - Helper functions
        - Test data generators
        
        Here's the code to test:
        \n{code}
    """
    return user

In [7]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [8]:
exec(pi)

Result: 3.141592658589
Execution Time: 58.228012 seconds


In [53]:
def messages_for_comment(python):
    return [
        {"role": "system", "content": system_prompt_for_comment()},
        {"role": "user", "content": user_prompt_for_comment(python)}
    ]

In [54]:
def messages_for_unit_test(python):
    return [
        {"role": "system", "content": system_prompt_for_unit_test()},
        {"role": "user", "content": user_prompt_for_unit_test(python)}
    ]

In [58]:
def stream_comment_gpt(code):
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for_comment(code), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        #print(fragment, end='', flush=True)
        yield reply.replace('```','') 
    

In [18]:
stream_comment_gpt(pi)

```python
import time

def calculate(iterations: int, param1: float, param2: float) -> float:
    """
    Performs a series of mathematical operations in a loop to calculate a result.

    This function iteratively modifies a result variable through a series of arithmetic
    operations. Essentially, it calculates the sum of alternating series adjustments,
    simulating a specific numerical approximation process.

    Args:
        iterations (int): The number of iterations to perform. Must be a positive integer.
        param1 (float): The factor applied for multiplication inside the iteration.
        param2 (float): The factor subtracted and added inside the iteration for denominator adjustment.

    Returns:
        float: The calculated result after completing all iterations.

    Raises:
        ZeroDivisionError: If any calculated denominator becomes zero during execution,
        which may happen if `i * param1 - param2` or `i * param1 + param2` evaluates to zero.

    Usage E

In [59]:
def stream_unit_test_gpt(code):
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for_unit_test(code), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        #print(fragment, end='', flush=True)
        yield reply.replace('```','')

In [73]:
stream_unit_test_gpt(pi)

<generator object stream_unit_test_gpt at 0x000002A31DC9C440>

In [60]:
def stream_comment_claude(code):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_prompt_for_comment(),
        messages=[{"role": "user", "content": user_prompt_for_comment(code)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            #print(text, end="", flush=True)
            yield reply.replace('```','')

In [64]:
def stream_unit_test_claude(code):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_prompt_for_unit_test(),
        messages=[{"role": "user", "content": user_prompt_for_unit_test(code)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            #print(text, end="", flush=True)
            yield reply.replace('```','')

In [23]:
stream_comment_claude(pi)

Here's the documented version of your Python code:

```python
import time

 float) -> float:rations: int, param1: float, param2:
    """
Calculates a series sum based on the given parameters.

    This function computes a series sum using the formula:
i*param1 + param2) - 1/(i*param1 - param2)) for i from 1 to iterations.

    Args:
 iterations to perform. Must be positive.
float): The first parameter used in the calculation.
(float): The second parameter used in the calculation.

    Returns:
.       float: The result of the series sum calculation

    Raises:
.       ValueError: If iterations is not positive
is 0 or if param2 is equal to param1.

    Example:
 = calculate(1000, 4, 1)
>>> print(f"{result:.6f}")
.392699 0

    Note:
 The function may be computationally expensive for large numbers of iterations.
, floating-point precision limitations may affect accuracy.
    """
    if iterations <= 0:
 must be a positive integer")rations

    result = 1.0
    for i in range(1, iteratio

In [63]:
stream_unit_test_claude(pi)

Here's a comprehensive set of unit tests for the given Python code using pytest:

```python
import pytest
import time
 import isclose
from unittest.mock import patch

# Import the function to be tested
# Assuming the code is in a file named your_module.py

# Test data generator
_data():rate_test
    return [
, 2, 1, 0.6931471805),
 3, 2, 0.6931471806),
, 3, 0.6931471806),
, 1, 0.6931471806),
    ]

 datature for common test
@pytest.fixture
def common_data():
return {
        'iterations': 100,
        'param1': 4,
        'param2': 1
    }

# Normal case tests
rize("iterations, param1, param2, expected", generate_test_data())
cases(iterations, param1, param2, expected):
1, param2) = calculate(iterations, param
(result, expected, rel_tol=1e-9), f"Expected {expected}, but got {result}"

 cases and boundary values
_cases():calculate_edge
d inputsst with minimum vali
 0) == 2.0 calculate(1, 1,
    
    # Test with very large iterations
_result = calculate(10**8, 4, 1)
, 0.6931471806, rel_t

In [40]:
def stream_comment_model(model, model_url, code):
    tokenizer = AutoTokenizer.from_pretrained(model)
    messages = messages_for_comment(code)
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    client = InferenceClient(model_url, token=hf_token)
    stream = client.text_generation(text, stream=True, details=True, max_new_tokens=3000)
    result = ""
    for r in stream:
        #print(r.token.text, end = "")
        result += r.token.text
        yield result    
    

In [67]:
def stream_unit_test_model(model, model_url, code):
    tokenizer = AutoTokenizer.from_pretrained(model)
    messages = messages_for_unit_test(code)
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    client = InferenceClient(model_url, token=hf_token)
    stream = client.text_generation(text, stream=True, details=True, max_new_tokens=3000)
    result = ""
    for r in stream:
        #print(r.token.text, end = "")
        result += r.token.text
        yield result    
    

In [41]:
stream_comment_model(code_qwen, CODE_QWEN_URL, pi)

<generator object stream_comment_model at 0x000002A31C666770>

In [70]:
stream_unit_test_model(code_qwen, CODE_QWEN_URL, pi)

Here are the unit tests for the given Python code:

```python
import pytest
import time
 unittest.mock import patch

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
 i * param1 - param2
result -= (1/j)
        j = i * param1 + param2
result += (1/j)
    return result

@pytest.fixture
 mock_time():
('time.time') as mock_time:
yield mock_time

_calculate_normal_inputs(mock_time):
mock_time.return_value = 0
result = calculate(100_000_000, 4, 1) * 4
expected_result = 0.0
 == expected_result

_calculate_edge_cases(mock_time):
    mock_time.return_value = 0
 calculate(0, 4, 1) * 4
    expected_result = 0.0
 result == expected_result

 = calculate(100_000_000, 0, 1) * 4
expected_result = 0.0
 result == expected_result

 = calculate(100_000_000, 4, 0) * 4
_result = 0.0
    assert result == expected_result

def test_calculate_invalid_inputs(mock_time):
    mock_time.return_value = 0
.raises(TypeError):
calculate(100_000_000, 'a', 1) * 4
with pyte

In [46]:
def comment_code(python, model):
    if model=="GPT":
        result = stream_comment_gpt(python)
    elif model=="Claude":
        result = stream_comment_claude(python)
    elif model=="CodeQwen":
        result = stream_comment_model(code_qwen, CODE_QWEN_URL, python)
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far     

In [69]:
def get_unit_test(python, model):
    if model=="GPT":
        result = stream_unit_test_gpt(python)
    elif model=="Claude":
        result = stream_unit_test_claude(python)
    elif model=="CodeQwen":
        result = stream_unit_test_model(code_qwen, CODE_QWEN_URL, python)
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far     

In [35]:
css = """
.python {background-color: #306998;}
.cpp {background-color: #050;}
"""

In [74]:
with gr.Blocks(css=css) as ui:
    gr.Markdown("## Convert code from Python to C++")
    with gr.Row():
        python = gr.Textbox(label="Python code:", value=pi, lines=10)
        result = gr.Textbox(label="Result code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude","CodeQwen"], label="Select model", value="GPT")
    with gr.Row():
        comment_button = gr.Button("Comment code")
    with gr.Row():
        unit_test_button = gr.Button("Unit Test code")
        
    comment_button.click(comment_code, inputs=[python, model], outputs=[result])
    unit_test_button.click(get_unit_test, inputs=[python, model], outputs=[result])
ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7881

To create a public link, set `share=True` in `launch()`.


Here are the unit tests for the given Python code:

```python
import pytest
import time
 unittest.mock import patch

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
 i * param1 - param2
        result -= (1/j)
 i * param1 + param2
result += (1/j)
    return result

@pytest.fixture
 mock_time():
    with patch('time.time') as mock_time:
ield mock_time

calculate_normal_inputs(mock_time):
time.return_value = 0
 calculate(100_000_000, 4, 1) * 4
result = 0.0_
assert result == expected_result

 test_calculate_edge_cases(mock_time):
time.return_value = 0
 calculate(0, 4, 1) * 4
_result = 0.0
    assert result == expected_result

    result = calculate(100_000_000, 0, 1) * 4
result = 0.0_
assert result == expected_result

result = calculate(100_000_000, 4, 0) * 4
    expected_result = 0.0
assert result == expected_result

 test_calculate_invalid_inputs(mock_time):
_time.return_value = 0
test.raises(TypeError):
        calculate(100_000_000, 'a'

Traceback (most recent call last):
  File "C:\Users\ebaba\.conda\envs\llms\Lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ebaba\.conda\envs\llms\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ebaba\.conda\envs\llms\Lib\site-packages\gradio\blocks.py", line 2047, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ebaba\.conda\envs\llms\Lib\site-packages\gradio\blocks.py", line 1606, in call_function
    prediction = await utils.async_iteration(iterator)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ebaba\.conda\envs\llms\Lib\site-packages\gradio\utils.py", line 714, in async_iteration
    return await anext(iterator)
           ^

Here are the unit tests for the given Python code:

```python
import pytest
import time
est.mock import patch

(iterations, param1, param2):
result = 1.0
for i in range(1, iterations+1):
j = i * param1 - param2
 -= (1/j)esult
j = i * param1 + param2
 += (1/j)esult
return result

pytest.fixture
_time():
    with patch('time.time') as mock_time:
ield mock_time

calculate_normal_inputs(mock_time):
time.return_value = 0
 calculate(100_000_000, 4, 1) * 4
_result = 0.0
    assert result == expected_result

def test_calculate_edge_cases(mock_time):
mock_time.return_value = 0
result = calculate(0, 4, 1) * 4
    expected_result = 0.0
 result == expected_result

 = calculate(100_000_000, 0, 1) * 4
_result = 0.0
assert result == expected_result

result = calculate(100_000_000, 4, 0) * 4
    expected_result = 0.0
 result == expected_result

_calculate_invalid_inputs(mock_time):
time.return_value = 0
    with pytest.raises(TypeError):
(100_000_000, 'a', 1) * 4
ises(TypeError):ra
ulate(100_000_000, 

Traceback (most recent call last):
  File "C:\Users\ebaba\.conda\envs\llms\Lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ebaba\.conda\envs\llms\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ebaba\.conda\envs\llms\Lib\site-packages\gradio\blocks.py", line 2047, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ebaba\.conda\envs\llms\Lib\site-packages\gradio\blocks.py", line 1606, in call_function
    prediction = await utils.async_iteration(iterator)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ebaba\.conda\envs\llms\Lib\site-packages\gradio\utils.py", line 714, in async_iteration
    return await anext(iterator)
           ^